# GDP PPP /Capita Vulnerability Indicator

Notebook environment to migrate worldbank csv file https://data.worldbank.org/indicator/NY.GDP.PCAP.CD to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
#%pip install tqdm

<IPython.core.display.Javascript object>

In [3]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math
from tqdm import tqdm 

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets"


# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)

<IPython.core.display.Javascript object>

In [4]:
# Project paths & files (manual input)
dataset_dir = pathlib.Path().joinpath(gca_data_dir,"03_Vulnerability","GDP")
dataset_dir_path = dataset_dir.joinpath("GDP_PPP_CAP_original.nc")
CF_dir = dataset_dir.joinpath("CF")  # directory to save output CF check files
template_path= pathlib.Path().joinpath(gca_data_dir,r"04_Auxillary_files\Arjen_Vector_Template")
dataset_dir_path

WindowsPath('P:/11209197-018-global-coastal-atlas/MSc_students/ClenmarRowe/Data/All_Datasets/Orig_Datasets/03_Vulnerability/GDP/GDP_PPP_CAP_original.nc')

<IPython.core.display.Javascript object>

In [5]:
df_template=pd.read_csv(template_path)
df_template

,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat
0,BOX_028_183_0,CHL,South America,Chile,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456
1,BOX_028_183_1,CHL,South America,Chile,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940
2,BOX_028_183_2,CHL,South America,Chile,-74.378628,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425
3,BOX_028_183_3,CHL,South America,Chile,-74.373950,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574
4,BOX_028_183_4,CHL,South America,Chile,-74.370425,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348
...,...,...,...,...,...,...,...,...,...,...
1739821,BOX_211_067_149,RUS,Europe,Russia,39.929937,64.701462,39.935198,64.698350,39.940460,64.695238
1739822,BOX_211_067_150,RUS,Europe,Russia,39.933577,64.702586,39.938839,64.699474,39.944100,64.696363
1739823,BOX_211_067_151,RUS,Europe,Russia,39.935546,64.703502,39.942003,64.700833,39.948460,64.698164
1739824,BOX_211_067_152,RUS,Europe,Russia,39.937050,64.704370,39.944697,64.702356,39.952343,64.700341


<IPython.core.display.Javascript object>

In [6]:

unique_temp=df_template["country_name"].unique()
unique_temp_code=df_template["country_id"].unique()
unique_temp_code

array(['CHL', 'unknown', 'ARG', 'FLK', 'SGS', 'ATF', 'NZL', 'URY', 'BRA',
       'SHN', 'ZAF', 'AUS', 'TON', 'FJI', 'PYF', 'COK', 'PER', 'NAM',
       'MOZ', 'MDG', 'FRA', 'MUS', 'NCL', 'VUT', 'WSM', 'ASM', 'WLF',
       'KIR', 'ECU', 'GAB', 'AGO', 'COG', 'COD', 'TZA', 'KEN', 'COM',
       'SOM', 'SYC', 'IOT', 'MDV', 'IDN', 'TLS', 'PNG', 'SLB', 'MEX',
       'GTM', 'SLV', 'PAN', 'COL', 'BLZ', 'HND', 'CRI', 'NIC', 'VEN',
       'GUY', 'TTO', 'KNA', 'SUR', 'NLD', 'ATG', 'MSR', 'CUW', 'LCA',
       'ABW', 'BRB', 'DOM', 'VCT', 'GRD', 'DMA', 'CPV', 'GHA', 'GNB',
       'SEN', 'LBR', 'GMB', 'SLE', 'GIN', 'CIV', 'MRT', 'NGA', 'CMR',
       'BEN', 'GNQ', 'TGO', 'SOL', 'SAU', 'ERI', 'YEM', 'DJI', 'OMN',
       'LKA', 'IND', 'MMR', 'MYS', 'SGP', 'VNM', 'THA', 'KHM', 'PHL',
       'BRN', 'PLW', 'FSM', 'MNP', 'GUM', 'MHL', 'USA', 'HTI', 'BHS',
       'CUB', 'JAM', 'TCA', 'CYM', 'VGB', 'PRI', 'VIR', 'BLM', 'AIA',
       'BMU', 'MAF', 'ESP', 'MAR', 'PRT', 'LBY', 'TUN', 'EGY', 'LBN',
       'ISR', 'J

<IPython.core.display.Javascript object>

In [7]:
# Data from other dataset input here
df_GDP=pd.read_csv(dataset_dir.joinpath("gdp_world\API_NY.GDP.PCAP.CD_DS2_en_csv_v2_6556290\API_NY.GDP.PCAP.CD_DS2_en_csv_v2_6556290.csv"),sep=",",skiprows=list(range(0,4)))
df_GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,26940.264114,28419.264534,28449.712946,29329.081747,30918.483584,31902.809818,24008.127822,29127.759384,33300.838819,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,141.385955,144.342434,148.774835,157.047580,166.849791,177.769086,...,1725.332959,1554.167299,1444.003514,1625.286236,1558.307482,1507.982881,1355.805923,1545.613215,1644.062829,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,...,626.512930,566.881133,523.053012,526.140801,492.090632,497.741429,512.055098,355.777826,NaN,NaN
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,107.053706,112.128417,117.814663,122.370114,130.700278,137.301801,...,2248.316255,1882.264038,1648.762676,1590.277754,1735.374911,1812.446822,1688.075575,1766.943618,1785.312219,NaN
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,5011.984412,3217.339244,1809.709377,2439.374441,2540.508878,2191.347764,1450.905112,1927.474078,3000.444231,NaN


<IPython.core.display.Javascript object>

In [8]:
# Original dataset dependent

#df_GDP=df_GDP[[df_GDP.columns[0],df_GDP.columns[1],df_GDP.columns[-3],df_GDP.columns[-4],df_GDP.columns[-5]]]
# country_list_RS=df_RS[country_column_RS]

df_GDP=df_GDP.drop(["Indicator Name","Indicator Code","Unnamed: 67"],axis=1)
df_GDP.head()


,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26514.868980,26940.264114,28419.264534,28449.712946,29329.081747,30918.483584,31902.809818,24008.127822,29127.759384,33300.838819
1,Africa Eastern and Southern,AFE,141.385955,144.342434,148.774835,157.047580,166.849791,177.769086,186.630486,199.559835,...,1736.849038,1725.332959,1554.167299,1444.003514,1625.286236,1558.307482,1507.982881,1355.805923,1545.613215,1644.062829
2,Afghanistan,AFG,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,143.103233,167.165675,...,638.733185,626.512930,566.881133,523.053012,526.140801,492.090632,497.741429,512.055098,355.777826,NaN
3,Africa Western and Central,AFW,107.053706,112.128417,117.814663,122.370114,130.700278,137.301801,143.012562,127.325268,...,2154.150832,2248.316255,1882.264038,1648.762676,1590.277754,1735.374911,1812.446822,1688.075575,1766.943618,1785.312219
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5061.349240,5011.984412,3217.339244,1809.709377,2439.374441,2540.508878,2191.347764,1450.905112,1927.474078,3000.444231


<IPython.core.display.Javascript object>

In [9]:
df_GDP["3_yr_Average"]= (df_GDP["2021"]+df_GDP["2020"]+df_GDP["2019"])/3
df_GDP.head()

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,3_yr_Average
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26940.264114,28419.264534,28449.712946,29329.081747,30918.483584,31902.809818,24008.127822,29127.759384,33300.838819,28346.232341
1,Africa Eastern and Southern,AFE,141.385955,144.342434,148.774835,157.047580,166.849791,177.769086,186.630486,199.559835,...,1725.332959,1554.167299,1444.003514,1625.286236,1558.307482,1507.982881,1355.805923,1545.613215,1644.062829,1469.800673
2,Afghanistan,AFG,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,143.103233,167.165675,...,626.512930,566.881133,523.053012,526.140801,492.090632,497.741429,512.055098,355.777826,NaN,455.191451
3,Africa Western and Central,AFW,107.053706,112.128417,117.814663,122.370114,130.700278,137.301801,143.012562,127.325268,...,2248.316255,1882.264038,1648.762676,1590.277754,1735.374911,1812.446822,1688.075575,1766.943618,1785.312219,1755.822005
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5011.984412,3217.339244,1809.709377,2439.374441,2540.508878,2191.347764,1450.905112,1927.474078,3000.444231,1856.575651


<IPython.core.display.Javascript object>

In [10]:
unique_GDP=df_GDP["Country Name"].unique()
unique_GDP_code=df_GDP["Country Code"].unique()
unique_GDP_code

array(['ABW', 'AFE', 'AFG', 'AFW', 'AGO', 'ALB', 'AND', 'ARB', 'ARE',
       'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL',
       'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ',
       'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN',
       'CEB', 'CHE', 'CHI', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG',
       'COL', 'COM', 'CPV', 'CRI', 'CSS', 'CUB', 'CUW', 'CYM', 'CYP',
       'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'EAP', 'EAR',
       'EAS', 'ECA', 'ECS', 'ECU', 'EGY', 'EMU', 'ERI', 'ESP', 'EST',
       'ETH', 'EUU', 'FCS', 'FIN', 'FJI', 'FRA', 'FRO', 'FSM', 'GAB',
       'GBR', 'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HIC', 'HKG', 'HND', 'HPC',
       'HRV', 'HTI', 'HUN', 'IBD', 'IBT', 'IDA', 'IDB', 'IDN', 'IDX',
       'IMN', 'IND', 'INX', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
       'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA',
       'KOR', 'KWT',

<IPython.core.display.Javascript object>

In [11]:
#Create a list of matching countries

good_list=[]

for i in range(0,len(unique_temp)):
    for j in range(0,len(unique_GDP)):
        if unique_temp[i]==unique_GDP[j]:
            good_list.append(unique_temp[i])
good_list
            
len(good_list)


137

<IPython.core.display.Javascript object>

In [12]:
#Create a missing list

bad_list_temp=set(unique_temp).difference(set(good_list))
bad_list_GDP=set(unique_GDP).difference(set(good_list))
len(bad_list_GDP)

129

<IPython.core.display.Javascript object>

In [13]:


#to dataframe
bad_list_GDP_eye=pd.DataFrame({"child mortality": list(bad_list_GDP)})
bad_list_temp_eye=pd.DataFrame({"Arjen Template2": list(bad_list_temp)})

#to excel for eye check
bad_list_GDP_eye.to_excel(dataset_dir.joinpath("eye_checker_GDP.xlsx"))
bad_list_temp_eye.to_excel(dataset_dir.joinpath("eye_checker_temp_code.xlsx"))

<IPython.core.display.Javascript object>

In [14]:
# Switch names that are similar but problematic

name_mapping = {

"Hong Kong SAR, China"	:"Hong Kong S.A.R.",
"Korea, Rep.":	"South Korea",
"St. Vincent and the Grenadines":	"Saint Vincent and the Grenadines",
"Bahamas, The":	"The Bahamas",
"Congo, Rep.":"Republic of Congo",
"Venezuela, RB"	:"Venezuela",
"St. Lucia":	"Saint Lucia",
"St. Kitts and Nevis":	"Saint Kitts and Nevis",
"Micronesia, Fed. Sts."	:"Federated States of Micronesia",
"Congo, Dem. Rep.":	"Democratic Republic of the Congo",
"Syrian Arab Republic":	"Syria",
"Cabo Verde":	"Cape Verde",
"Gambia, The":	"Gambia",
"Egypt, Arab Rep.":	"Egypt",
"Turkiye":	"Turkey",
"Virgin Islands (U.S.)"	:"United States Virgin Islands",
"Viet Nam"	:"Vietnam",
"Cote d'Ivoire"	:"Ivory Coast",
"Korea, Dem. People's Rep."	:"North Korea",
"United States":"United States of America",
"Iran, Islamic Rep."	:"Iran",
"Brunei Darussalam"	:"Brunei",
"Yemen, Rep."	:"Yemen",
"Tanzania"	:"United Republic of Tanzania",
"St. Martin (French part)"	:"Saint Martin",
"Sint Maarten (Dutch part)"	:"Saint Martin",
"Guinea-Bissau":"Guinea Bissau",
"Russian Federation"	:"Russia"


}

<IPython.core.display.Javascript object>

In [15]:
df_GDP["Country Name"]=df_GDP["Country Name"].replace(name_mapping)
unique_GDP_new= df_GDP["Country Name"].unique()
unique_GDP_new

array(['Aruba', 'Africa Eastern and Southern', 'Afghanistan',
       'Africa Western and Central', 'Angola', 'Albania', 'Andorra',
       'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia',
       'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'The Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada',
       'Central Europe and the Baltics', 'Switzerland', 'Channel Islands',
       'Chile', 'China', 'Ivory Coast', 'Cameroon',
       'Democratic Republic of the Congo', 'Republic of Congo',
       'Colombia', 'Comoros', 'Cape Verde', 'Costa Rica',
       'Caribbean small states', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic

<IPython.core.display.Javascript object>

In [16]:
#Create a list of matching countries

good_list=[]

for i in range(0,len(unique_temp)):
    for j in range(0,len(unique_GDP_new)):
        if unique_temp[i]==unique_GDP_new[j]:
            good_list.append(unique_temp[i])
good_list
            
len(good_list)


164

<IPython.core.display.Javascript object>

In [17]:
#Create a missing list

bad_list_temp=set(unique_temp).difference(set(good_list))
bad_list_GDP=set(unique_GDP_new).difference(set(good_list))
len(bad_list_GDP)

101

<IPython.core.display.Javascript object>

In [18]:
df_GDP.head()

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,3_yr_Average
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26940.264114,28419.264534,28449.712946,29329.081747,30918.483584,31902.809818,24008.127822,29127.759384,33300.838819,28346.232341
1,Africa Eastern and Southern,AFE,141.385955,144.342434,148.774835,157.047580,166.849791,177.769086,186.630486,199.559835,...,1725.332959,1554.167299,1444.003514,1625.286236,1558.307482,1507.982881,1355.805923,1545.613215,1644.062829,1469.800673
2,Afghanistan,AFG,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,143.103233,167.165675,...,626.512930,566.881133,523.053012,526.140801,492.090632,497.741429,512.055098,355.777826,NaN,455.191451
3,Africa Western and Central,AFW,107.053706,112.128417,117.814663,122.370114,130.700278,137.301801,143.012562,127.325268,...,2248.316255,1882.264038,1648.762676,1590.277754,1735.374911,1812.446822,1688.075575,1766.943618,1785.312219,1755.822005
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5011.984412,3217.339244,1809.709377,2439.374441,2540.508878,2191.347764,1450.905112,1927.474078,3000.444231,1856.575651


<IPython.core.display.Javascript object>

In [19]:
# df_GDP=df_GDP[["Country Name","3_yr_Average"]]
# df_GDP

<IPython.core.display.Javascript object>

In [20]:
df_template.head()
# len(df_template)

,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat
0,BOX_028_183_0,CHL,South America,Chile,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456
1,BOX_028_183_1,CHL,South America,Chile,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940
2,BOX_028_183_2,CHL,South America,Chile,-74.378628,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425
3,BOX_028_183_3,CHL,South America,Chile,-74.373950,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574
4,BOX_028_183_4,CHL,South America,Chile,-74.370425,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348


<IPython.core.display.Javascript object>

In [21]:


# Merge df_template with RS_counter on the "Country" column
merged_df = pd.merge(df_template, df_GDP, left_on="country_name", right_on="Country Name", how="left")

# Rename the merged column to "Recent_Shocks (Deaths)"
merged_df.rename(columns={"3_yr_Average": "GDP_PPP_Cap (3_yr_Average)"}, inplace=True)

# Fill NAs with "N/A" in the "Recent_Shocks (Deaths)" column
#merged_df["Child Mortality Rate (3_yr_Average)"] = merged_df["Child Mortality Rate (3_yr_Average))"].fillna(0).astype('int64')
#merged_df["GDP_PPP_Cap (3_yr_Average)"] = merged_df["GDP_PPP_Cap (3_yr_Average)"].fillna(-9999).astype('int64')

#make all nans -9999 because of json dumps issue in geojson
merged_df=merged_df.fillna(0)#.astype('int64')
# Now merged_df_RS is the DataFrame with the appended total death data
merged_df=merged_df.drop("Country Name",axis=1)
merged_df


,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,GDP_PPP_Cap (3_yr_Average)
0,BOX_028_183_0,CHL,South America,Chile,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
1,BOX_028_183_1,CHL,South America,Chile,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
2,BOX_028_183_2,CHL,South America,Chile,-74.378628,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
3,BOX_028_183_3,CHL,South America,Chile,-74.373950,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
4,BOX_028_183_4,CHL,South America,Chile,-74.370425,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739914,BOX_211_067_149,RUS,Europe,Russia,39.929937,64.701462,39.935198,64.698350,39.940460,64.695238,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992
1739915,BOX_211_067_150,RUS,Europe,Russia,39.933577,64.702586,39.938839,64.699474,39.944100,64.696363,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992
1739916,BOX_211_067_151,RUS,Europe,Russia,39.935546,64.703502,39.942003,64.700833,39.948460,64.698164,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992
1739917,BOX_211_067_152,RUS,Europe,Russia,39.937050,64.704370,39.944697,64.702356,39.952343,64.700341,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992


<IPython.core.display.Javascript object>

In [22]:
merged_df.to_csv(dataset_dir.joinpath("GDP_Cap_mapped_to_transect.csv"),index=False)

<IPython.core.display.Javascript object>

In [23]:
df=pd.read_csv(dataset_dir.joinpath("GDP_Cap_mapped_to_transect.csv"))
df

,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,GDP_PPP_Cap (3_yr_Average)
0,BOX_028_183_0,CHL,South America,Chile,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
1,BOX_028_183_1,CHL,South America,Chile,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
2,BOX_028_183_2,CHL,South America,Chile,-74.378628,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
3,BOX_028_183_3,CHL,South America,Chile,-74.373950,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
4,BOX_028_183_4,CHL,South America,Chile,-74.370425,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348,...,14675.150873,13567.357217,13788.240016,15034.058425,15820.033357,14632.690308,13173.784794,16240.607776,15355.479740,14682.360959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739914,BOX_211_067_149,RUS,Europe,Russia,39.929937,64.701462,39.935198,64.698350,39.940460,64.695238,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992
1739915,BOX_211_067_150,RUS,Europe,Russia,39.933577,64.702586,39.938839,64.699474,39.944100,64.696363,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992
1739916,BOX_211_067_151,RUS,Europe,Russia,39.935546,64.703502,39.942003,64.700833,39.948460,64.698164,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992
1739917,BOX_211_067_152,RUS,Europe,Russia,39.937050,64.704370,39.944697,64.702356,39.952343,64.700341,...,14095.646484,9313.021484,8704.894531,10720.332031,11287.354492,11536.258789,10194.441406,12532.050781,15270.706055,11420.916992


<IPython.core.display.Javascript object>

In [24]:
# Convert the pandas dataframe to an xarray dataset
ds = xr.Dataset.from_dataframe(df)
ds


<xarray.Dataset>
Dimensions:                     (index: 1739919)
Coordinates:
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
Data variables: (12/75)
    transect_id                 (index) object 'BOX_028_183_0' ... 'BOX_211_0...
    country_id                  (index) object 'CHL' 'CHL' 'CHL' ... 'RUS' 'RUS'
    continent                   (index) object 'South America' ... 'Europe'
    country_name                (index) object 'Chile' 'Chile' ... 'Russia'
    Start_lon                   (index) float64 -74.39 -74.38 ... 39.94 39.94
    Start_lat                   (index) float64 -50.38 -50.38 ... 64.7 64.7
    ...                          ...
    2018                        (index) float64 1.582e+04 ... 1.129e+04
    2019                        (index) float64 1.463e+04 ... 1.154e+04
    2020                        (index) float64 1.317e+04 ... 1.019e+04
    2021                        (index) float64 1.624e+04 ... 1.253e+04
    2022                        (index) float64 1.536e+04 ... 1.527e+04
    GDP_PPP_Cap (3_yr_Average)  (index) float64 1.468e+04 ... 1.142e+04

<IPython.core.display.Javascript object>

In [25]:

os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
# Write the xarray dataset to a netCDF file
ds.to_netcdf(dataset_dir_path)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [26]:
# open datasets
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:                     (index: 1739919)
Coordinates:
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
Data variables: (12/75)
    transect_id                 (index) <U15 ...
    country_id                  (index) <U7 ...
    continent                   (index) <U23 ...
    country_name                (index) <U40 ...
    Start_lon                   (index) float64 ...
    Start_lat                   (index) float64 ...
    ...                          ...
    2018                        (index) float64 ...
    2019                        (index) float64 ...
    2020                        (index) float64 ...
    2021                        (index) float64 ...
    2022                        (index) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (index) float64 ...

<IPython.core.display.Javascript object>

In [27]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile= dataset_dir_path, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [28]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\03_Vulnerability\\GDP\\GDP_PPP_CAP_original.nc': {'warnings': '148', 'errors': '5'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [29]:
# open original datasets
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:                     (index: 1739919)
Coordinates:
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
Data variables: (12/75)
    transect_id                 (index) <U15 ...
    country_id                  (index) <U7 ...
    continent                   (index) <U23 ...
    country_name                (index) <U40 ...
    Start_lon                   (index) float64 ...
    Start_lat                   (index) float64 ...
    ...                          ...
    2018                        (index) float64 ...
    2019                        (index) float64 ...
    2020                        (index) float64 ...
    2021                        (index) float64 ...
    2022                        (index) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (index) float64 ...

<IPython.core.display.Javascript object>

In [30]:
#Check columns to know which years to get
df.columns

Index(['transect_id', 'country_id', 'continent', 'country_name', 'Start_lon',
       'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon', 'End_lat',
       'Country Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966',
       '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
       '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', 'GDP_PPP_Cap (3_yr_Average)'],
      dtype='object')

<IPython.core.display.Javascript object>

In [31]:
df.iloc[:,11:-1].columns

Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

<IPython.core.display.Javascript object>

In [32]:
#make a dataframe of all the times and convert them to a numpy array
df_times=(df.iloc[:,11:-1].values)
df_times.shape


(1739919, 63)

<IPython.core.display.Javascript object>

In [33]:
# Define the start and end years
start_year =  1960
end_year =  2022

# Use date_range to create a DatetimeIndex with yearly frequency
date_range = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-01-01', freq='AS')

# Convert the DatetimeIndex to a list if needed
date_list = date_range.tolist()
len(date_list)

63

<IPython.core.display.Javascript object>

In [34]:
ds_temporal=ds
ds_temporal

<xarray.Dataset>
Dimensions:                     (index: 1739919)
Coordinates:
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
Data variables: (12/75)
    transect_id                 (index) <U15 ...
    country_id                  (index) <U7 ...
    continent                   (index) <U23 ...
    country_name                (index) <U40 ...
    Start_lon                   (index) float64 ...
    Start_lat                   (index) float64 ...
    ...                          ...
    2018                        (index) float64 ...
    2019                        (index) float64 ...
    2020                        (index) float64 ...
    2021                        (index) float64 ...
    2022                        (index) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (index) float64 ...

<IPython.core.display.Javascript object>

In [36]:
# combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = ds_temporal["Start_lon"].values
start_lats = ds_temporal["Start_lat"].values
end_lons = ds_temporal["End_lon"].values
end_lats = ds_temporal["End_lat"].values
coords_temporal = zip(zip(start_lons, start_lats), zip(end_lons, end_lats))

ds_temporal["transect_geom"] = (
    ["index"],
    [str(LineString(line)) for line in coords_temporal],
)
ds_temporal["transect_geom"].attrs["long_name"] = "Transect Geometry"

<IPython.core.display.Javascript object>

In [37]:

# set some data variables to coordinates to *avoid duplication* of dimensions in later stage
ds_temporal = ds_temporal.set_coords(['transect_id', 'country_name', 'continent', 'Intersect_lon', 'Intersect_lat', 'transect_geom','GDP_PPP_Cap (3_yr_Average)'])
ds_temporal

<xarray.Dataset>
Dimensions:                     (index: 1739919)
Coordinates:
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
    transect_id                 (index) <U15 ...
    continent                   (index) <U23 ...
    country_name                (index) <U40 ...
    Intersect_lon               (index) float64 ...
    Intersect_lat               (index) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (index) float64 ...
    transect_geom               (index) <U92 'LINESTRING (-74.3863095545 -50....
Data variables: (12/69)
    country_id                  (index) <U7 ...
    Start_lon                   (index) float64 -74.39 -74.38 ... 39.94 39.94
    Start_lat                   (index) float64 -50.38 -50.38 ... 64.7 64.7
    End_lon                     (index) float64 -74.4 -74.39 ... 39.95 39.95
    End_lat                     (index) float64 -50.39 -50.39 ... 64.7 64.71
    Country Code                (index) <U3 ...
    ...                          ...
    2017                        (index) float64 ...
    2018                        (index) float64 ...
    2019                        (index) float64 ...
    2020                        (index) float64 ...
    2021                        (index) float64 ...
    2022                        (index) float64 ...

<IPython.core.display.Javascript object>

In [38]:
keep_vars = []
allvars = list(ds_temporal.keys())
delete_vars = list(set(allvars).difference(set(keep_vars)))

ds_temporal = ds_temporal.drop_vars(delete_vars)
ds_temporal

<xarray.Dataset>
Dimensions:                     (index: 1739919)
Coordinates:
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
    transect_id                 (index) <U15 ...
    continent                   (index) <U23 ...
    country_name                (index) <U40 ...
    Intersect_lon               (index) float64 ...
    Intersect_lat               (index) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (index) float64 ...
    transect_geom               (index) <U92 'LINESTRING (-74.3863095545 -50....
Data variables:
    *empty*

<IPython.core.display.Javascript object>

In [39]:
# Once coordinates are set, we can add another time dimension that will not duplicate 
ds_temporal=ds_temporal.expand_dims(dim={"time": date_list})
ds_temporal


<xarray.Dataset>
Dimensions:                     (time: 63, index: 1739919)
Coordinates:
  * time                        (time) datetime64[ns] 1960-01-01 ... 2022-01-01
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
    transect_id                 (index) <U15 ...
    continent                   (index) <U23 ...
    country_name                (index) <U40 ...
    Intersect_lon               (index) float64 ...
    Intersect_lat               (index) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (index) float64 ...
    transect_geom               (index) <U92 'LINESTRING (-74.3863095545 -50....
Data variables:
    *empty*

<IPython.core.display.Javascript object>

In [40]:
# assign a new variable that canges both in time and in location(country)
ds_reshaped=ds_temporal.assign(GDP_PPP_Cap=(["index","time"],df_times))
ds_reshaped

<xarray.Dataset>
Dimensions:                     (time: 63, index: 1739919)
Coordinates:
  * time                        (time) datetime64[ns] 1960-01-01 ... 2022-01-01
  * index                       (index) int64 0 1 2 ... 1739916 1739917 1739918
    transect_id                 (index) <U15 ...
    continent                   (index) <U23 ...
    country_name                (index) <U40 ...
    Intersect_lon               (index) float64 ...
    Intersect_lat               (index) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (index) float64 ...
    transect_geom               (index) <U92 'LINESTRING (-74.3863095545 -50....
Data variables:
    GDP_PPP_Cap                 (index, time) float64 504.8 580.9 ... 1.527e+04

<IPython.core.display.Javascript object>

In [41]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds = ds_reshaped.rename_dims({"index": "nstations"})
ds


<xarray.Dataset>
Dimensions:                     (time: 63, nstations: 1739919)
Coordinates:
  * time                        (time) datetime64[ns] 1960-01-01 ... 2022-01-01
  * index                       (nstations) int64 0 1 2 ... 1739917 1739918
    transect_id                 (nstations) <U15 ...
    continent                   (nstations) <U23 ...
    country_name                (nstations) <U40 ...
    Intersect_lon               (nstations) float64 ...
    Intersect_lat               (nstations) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (nstations) float64 ...
    transect_geom               (nstations) <U92 'LINESTRING (-74.3863095545 ...
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap                 (nstations, time) float64 504.8 ... 1.527e+04

<IPython.core.display.Javascript object>

In [42]:
#ds=ds.drop_vars(["index"])
ds = ds.rename_vars( {"index": "stations" })
ds

<xarray.Dataset>
Dimensions:                     (time: 63, nstations: 1739919)
Coordinates:
  * time                        (time) datetime64[ns] 1960-01-01 ... 2022-01-01
  * stations                    (nstations) int64 0 1 2 ... 1739917 1739918
    transect_id                 (nstations) <U15 ...
    continent                   (nstations) <U23 ...
    country_name                (nstations) <U40 ...
    Intersect_lon               (nstations) float64 ...
    Intersect_lat               (nstations) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (nstations) float64 ...
    transect_geom               (nstations) <U92 'LINESTRING (-74.3863095545 ...
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap                 (nstations, time) float64 504.8 ... 1.527e+04

<IPython.core.display.Javascript object>

In [43]:
import json

# NetCDF attribute alterations
f_global = open(dataset_dir.joinpath("metadata_GDP_Cap.json"))
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"
ds

<xarray.Dataset>
Dimensions:                     (time: 63, nstations: 1739919)
Coordinates:
  * time                        (time) datetime64[ns] 1960-01-01 ... 2022-01-01
  * stations                    (nstations) int64 0 1 2 ... 1739917 1739918
    transect_id                 (nstations) <U15 ...
    continent                   (nstations) <U23 ...
    country_name                (nstations) <U40 ...
    Intersect_lon               (nstations) float64 ...
    Intersect_lat               (nstations) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (nstations) float64 ...
    transect_geom               (nstations) <U92 'LINESTRING (-74.3863095545 ...
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap                 (nstations, time) float64 504.8 ... 1.527e+04
Attributes: (12/21)
    TITLE:               GDP per Capita
    TITLE_ABBREVIATION:  GDPPC
    DESCRIPTION:         GDP per capita is an economic metric that breaks dow...
    SHORT_DESCRIPTION:   GDP per capita is an economic measure that indicates...
    INSTITUTION:         World Bank
    PROVIDERS:           {"name": "World Bank", "url": "https://www.worldbank...
    ...                  ...
    DOI:                 
    LONG_NAME:           GDP per capita
    UNITS:               current US$
    COMMENT:             An additional coordinate titled 3 year average is ad...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [44]:
# change dtypes from unsigned to signed
object_vars = ['transect_id', 'country_name', 'continent', 'transect_geom']
for i in object_vars:
    ds[i] = ds[i].astype('S')
ds

<xarray.Dataset>
Dimensions:                     (time: 63, nstations: 1739919)
Coordinates:
  * time                        (time) datetime64[ns] 1960-01-01 ... 2022-01-01
  * stations                    (nstations) int64 0 1 2 ... 1739917 1739918
    transect_id                 (nstations) |S15 b'BOX_028_183_0' ... b'BOX_2...
    continent                   (nstations) |S23 b'South America' ... b'Europe'
    country_name                (nstations) |S40 b'Chile' b'Chile' ... b'Russia'
    Intersect_lon               (nstations) float64 ...
    Intersect_lat               (nstations) float64 ...
    GDP_PPP_Cap (3_yr_Average)  (nstations) float64 ...
    transect_geom               (nstations) |S92 b'LINESTRING (-74.3863095545...
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap                 (nstations, time) float64 504.8 ... 1.527e+04
Attributes: (12/21)
    TITLE:               GDP per Capita
    TITLE_ABBREVIATION:  GDPPC
    DESCRIPTION:         GDP per capita is an economic metric that breaks dow...
    SHORT_DESCRIPTION:   GDP per capita is an economic measure that indicates...
    INSTITUTION:         World Bank
    PROVIDERS:           {"name": "World Bank", "url": "https://www.worldbank...
    ...                  ...
    DOI:                 
    LONG_NAME:           GDP per capita
    UNITS:               current US$
    COMMENT:             An additional coordinate titled 3 year average is ad...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [45]:

# rename variables, if necessary
ds = ds.rename_vars(
       {"Intersect_lon": "lon", "Intersect_lat": "lat",
        "country_name": "country" ,
        "GDP_PPP_Cap (3_yr_Average)" : "gdp_ppp_cap_3_yr"
       }
  )
ds

<xarray.Dataset>
Dimensions:           (time: 63, nstations: 1739919)
Coordinates:
  * time              (time) datetime64[ns] 1960-01-01 1961-01-01 ... 2022-01-01
  * stations          (nstations) int64 0 1 2 3 ... 1739916 1739917 1739918
    transect_id       (nstations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    continent         (nstations) |S23 b'South America' ... b'Europe'
    country           (nstations) |S40 b'Chile' b'Chile' ... b'Russia' b'Russia'
    lon               (nstations) float64 ...
    lat               (nstations) float64 ...
    gdp_ppp_cap_3_yr  (nstations) float64 ...
    transect_geom     (nstations) |S92 b'LINESTRING (-74.3863095545 -50.37765...
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap       (nstations, time) float64 504.8 580.9 ... 1.527e+04
Attributes: (12/21)
    TITLE:               GDP per Capita
    TITLE_ABBREVIATION:  GDPPC
    DESCRIPTION:         GDP per capita is an economic metric that breaks dow...
    SHORT_DESCRIPTION:   GDP per capita is an economic measure that indicates...
    INSTITUTION:         World Bank
    PROVIDERS:           {"name": "World Bank", "url": "https://www.worldbank...
    ...                  ...
    DOI:                 
    LONG_NAME:           GDP per capita
    UNITS:               current US$
    COMMENT:             An additional coordinate titled 3 year average is ad...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [46]:


# add or change certain variable / coordinate attributes
### dataset attributes is a dictionary of dictionaries
dataset_attributes = {
     "stations": { "long_name": "stations", "units": "1"},
     "lon": {"standard_name": "longitude", "long_name": "longitude", "units": "degrees_east"},
    "lat": {"standard_name": "latitude", "long_name": "latitude", "units": "degrees_north"},
    "transect_id": { "long_name": "Transect Identity", "units": "1"},
    "continent": { "long_name": "Continent", "units": "1"},
    "country": { "long_name": "Country", "units": "1"},
    "GDP_PPP_Cap": { "long_name": "GDP per capita", "units": "1"},
    "gdp_ppp_cap_3_yr": { "long_name": "GDP per capita (3 year Average)", "units": "1"},
    "time": { "long_name": "julian day (UT)"},
}  # specify custom (CF convention) attributes

 # add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        ds[k].attrs = dataset_attributes[k]
    except:
        continue


ds

<xarray.Dataset>
Dimensions:           (time: 63, nstations: 1739919)
Coordinates:
  * time              (time) datetime64[ns] 1960-01-01 1961-01-01 ... 2022-01-01
  * stations          (nstations) int64 0 1 2 3 ... 1739916 1739917 1739918
    transect_id       (nstations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    continent         (nstations) |S23 b'South America' ... b'Europe'
    country           (nstations) |S40 b'Chile' b'Chile' ... b'Russia' b'Russia'
    lon               (nstations) float64 ...
    lat               (nstations) float64 ...
    gdp_ppp_cap_3_yr  (nstations) float64 ...
    transect_geom     (nstations) |S92 b'LINESTRING (-74.3863095545 -50.37765...
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap       (nstations, time) float64 504.8 580.9 ... 1.527e+04
Attributes: (12/21)
    TITLE:               GDP per Capita
    TITLE_ABBREVIATION:  GDPPC
    DESCRIPTION:         GDP per capita is an economic metric that breaks dow...
    SHORT_DESCRIPTION:   GDP per capita is an economic measure that indicates...
    INSTITUTION:         World Bank
    PROVIDERS:           {"name": "World Bank", "url": "https://www.worldbank...
    ...                  ...
    DOI:                 
    LONG_NAME:           GDP per capita
    UNITS:               current US$
    COMMENT:             An additional coordinate titled 3 year average is ad...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [47]:
# Write the xarray dataset to a netCDF file
#Compliant netcdf
dataset_dir_path_CF=str(dataset_dir_path).replace("original","final")


ds.to_netcdf(path=dataset_dir_path_CF)

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [48]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path_CF)

# check original dataset
ds

<xarray.Dataset>
Dimensions:           (time: 63, nstations: 1739919)
Coordinates:
  * time              (time) datetime64[ns] 1960-01-01 1961-01-01 ... 2022-01-01
    stations          (nstations) int64 ...
    transect_id       (nstations) |S15 ...
    continent         (nstations) |S23 ...
    country           (nstations) |S40 ...
    lon               (nstations) float64 ...
    lat               (nstations) float64 ...
    gdp_ppp_cap_3_yr  (nstations) float64 ...
    transect_geom     (nstations) |S92 ...
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap       (nstations, time) float64 ...
Attributes: (12/21)
    TITLE:               GDP per Capita
    TITLE_ABBREVIATION:  GDPPC
    DESCRIPTION:         GDP per capita is an economic metric that breaks dow...
    SHORT_DESCRIPTION:   GDP per capita is an economic measure that indicates...
    INSTITUTION:         World Bank
    PROVIDERS:           {"name": "World Bank", "url": "https://www.worldbank...
    ...                  ...
    DOI:                 
    LONG_NAME:           GDP per capita
    UNITS:               current US$
    COMMENT:             An additional coordinate titled 3 year average is ad...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [49]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [50]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\03_Vulnerability\\GDP\\GDP_PPP_CAP_final.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [51]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(str(dataset_dir_path).replace("original","final").replace(".nc", ".zarr"), mode="w")

<IPython.core.display.Javascript object>